In [1]:
# function used to run the notebook
import pandas as pd
import functions.common as cf
import functions.plotting as pf
import xarray as xr
from IPython.display import Image
from IPython.core.display import HTML 

**Notebook Purpose:** 
- How to check the review data files for missing location coordinates and pressure?
  
###### Location Review Tests:
- **Pressure Comparison:**
    - Compare deployment depth to maximum pressure in the file
      
- **Missing Coordinates:** 
  - Check file coordinates against expected coordinates [ obs, time, lat, lon, pressure ]

**1**
<p style="color:green; font-size:1.1em;">Define Variables</p>

- Datasets Review List (local file created by 2.0_define_datareview_list.ipynb)
- Instrument Info: http://datareview.marine.rutgers.edu/instruments/view/

In [2]:
# Datasets Review List
reviewlist = pd.read_csv('define_datareview_list_CP05MOAS-GL335-05-PARADM000.csv')
reviewlist.index = reviewlist['Unnamed: 0'].values
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(reviewlist)[['datasets']]

# Instrument Info
col = list(reviewlist.columns)
refdes = reviewlist[col[1]][0].split('/')[-1].split('_')[1][0:27]
dr_data = cf.refdes_datareview_json(refdes)

df

,datasets
deployment0001,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190509T131304-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0001_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20141006T202152.905850-20141213T073238.247380.nc
deployment0002,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190509T131304-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0002_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20151014T001900.237980-20151110T091855.472810.nc
deployment0003,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190509T131304-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0003_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20160404T185705.311220-20160417T235956.145260.nc
deployment0004,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190509T131304-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0004_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20160529T204727.075500-20160626T091401.747920.nc
deployment0005,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190509T131304-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0005_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20170116T150223.595370-20170304T093047.153350.nc


**2**
- Pressure Comparison
    - <p style="color:blue">Deploy Depth</p>
    - source: https://github.com/ooi-integration/asset-management/tree/master/deployment
            - Instrument deployment depth defined in OOI Asset Management system
        
    - <p style="color:blue">Max Pressure</p>
    - source: datafiles in the review list
            - maximum pressure calculated from data file after eliminating data outside of global ranges and outliers (3 standard deviations).

In [6]:
df = pd.DataFrame()
for ii in range(len(reviewlist)):         
    deploy_num = int(reviewlist[col[0]][ii].split('t')[-1])
    deploy_info = cf.get_deployment_information(dr_data, deploy_num)
    deploy_depth = deploy_info['deployment_depth']
    
    # Check deployment pressure from asset management against pressure variable in file
    ds = xr.open_dataset(reviewlist[col[1]][ii], mask_and_scale=False)
    ds = ds.swap_dims({'obs': 'time'})
    press = pf.pressure_var(ds, list(ds.coords.keys()))
    if press is None:
        press = pf.pressure_var(ds, list(ds.data_vars.keys()))  
        print(press)
    pressure_compare, pressure_max, pressure_mean = cf.calculate_mean_pressure(press, ds, refdes, deploy_depth)
        
    df0 = pd.DataFrame({
                        'method': reviewlist[col[2]][ii],
                        'deploy_depth': [deploy_depth],
                        'data_pressure_max': [pressure_max]
                        }, index=[deploy_num])
    df = df.append(df0)
df

,method,deploy_depth,data_pressure_max
1,recovered_host,200,195.92
2,recovered_host,200,176.89
3,recovered_host,200,205.04
4,recovered_host,200,193.72
5,recovered_host,200,194.41


**3**
- Missing Coordinates
    - <p style="color:blue">check the coordinates in the files against:</p>
    
             ['obs', 'time', 'pressure', 'lat', 'lon']

In [7]:
df = pd.DataFrame()
for ii in range(len(reviewlist)):         
    deploy_num = int(reviewlist[col[0]][ii].split('t')[-1]) 
    ds = xr.open_dataset(reviewlist[col[1]][ii], mask_and_scale=False)
    ds = ds.swap_dims({'obs': 'time'})
    
    # check coordinate
    file_coordinates, coord_test = cf.check_coordinate(ds,refdes)
        
    df0 = pd.DataFrame({
                        'method': reviewlist[col[2]][ii],
                        'file_cord_list': [file_coordinates],
                        'cord_test':[coord_test]
                       }, index=[deploy_num])
    df = df.append(df0)
df

,method,file_cord_list,cord_test
1,recovered_host,"[obs, time, lon, lat, int_ctd_pressure]",pass
2,recovered_host,"[obs, time, lat, lon, int_ctd_pressure]",pass
3,recovered_host,"[obs, time, lat, lon, int_ctd_pressure]",pass
4,recovered_host,"[obs, time, lat, lon, int_ctd_pressure]",pass
5,recovered_host,"[obs, time, lat, lon, int_ctd_pressure]",pass


**4**

- <p style="color:green; font-size:1.1em;"> Glider Track<p>
    <p style="color:blue">check the glider track for drifts and other annomalies</p>

**Plotting script link:**
https://github.com/ooi-data-lab/data-review-tools/blob/master/plotting/scripts/mobile_assets/map_gliders.py  

In [46]:
Image(url= "https://marine.rutgers.edu/cool/ooi/data-eval/data_review/CP/CP05MOAS/CP05MOAS-GL335/CP05MOAS-GL335-05-PARADM000/CP05MOAS-GL335-05-PARADM000_glider_track.png")

**Note**
- The glider looks like it had steering issues. The dataset should be annotated if the glider didn't sample according to plan.

**END**

- Visit the instrument report page for more information:

(https://datareview.marine.rutgers.edu/instruments/report/CP05MOAS-GL335-05-PARADM000)